In [1]:
import pymysql
from sqlalchemy import create_engine

import pandas as pd
import time
import os

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium.webdriver.common.by import By

import datetime

# 1.엑셀에서 검색할 도시 정보 불러오기

In [2]:
Desktop = os.path.dirname(os.getcwd())
Desktop

'/Users/mycelebs/Desktop/staypia_데이터수집'

In [3]:
file_path=f'{Desktop}/data/staypia_bucketlist_v2.xlsx'
data = pd.read_excel(file_path)
data

cd_idx                                            bd_name  \
0    3193035  Take a walk in Harry Potter's magician street,...   
1    3193036  Explore the American Museum of Natural History...   
2    3193037  Take a walk to Kualoa Ranch, the primitive env...   
3    3193038  Relax by yourself in luxury and antique Plaza ...   
4    3193039  Read books in a romantic city of literature, N...   
..       ...                                                ...   
725  3664671  Sakte at the ‘Wollman Rink’, the ice-rink in t...   
726  3664673   Relax at the temple of happiness 'Punakha Dzong’   
727  3664679  Feel the thrill of crossing the sky! Ride an a...   
728  3664680  Take a picture at ‘Sirao Garden’, the beautifu...   
729        ↵  판단 잎으로 만드는 달콤함과 고소함의 결정체, 카야잼을 아시나요? 싱가포르의 '야쿤...   

                                     bd_name_ko  relation_cd_idx  \
0                <해리포터>의 마법사 거리, '다이애건 앨리' 걸어보기        1213719.0   
1         인류와 생물 그리고 우주에 관하여, 뉴욕 '자연사 박물관' 탐방하기        1228306.0   
2                 원시 자연의 모습 그대로, '쿠알로아 랜치' 산책하기        1218193.0   
3    100년의 역사를 간직한 호화로운 '플라자 호텔'에서 나 홀로 호캉스 즐기기        1228306.0   
4                   로맨틱한 문학 도시 '노팅힐'의 서점에서 독서하기        1213719.0   
..                                          ...              ...   
725                빌딩 숲속의 빙판, '울먼 링크'에서 스케이트 타기        1228306.0   
726          행복을 전하는 사원, 'Punakha Dzong'에서 힐링하기        3132116.0   
727      하늘을 가로지르는 짜릿함! '핑동 어드벤처 플라이트'에서 항공기 타기        1213328.0   
728                아름다운 하늘 정원, '시라오 가든'에서 사진 찍기        1215658.0   
729                                         NaN              NaN   

    related_vertical_name bd_category  ko_youtube  
0                      런던      TRAVEL           1  
1                      뉴욕      TRAVEL           1  
2                    호놀룰루      TRAVEL           1  
3                      뉴욕      TRAVEL           1  
4                      런던      TRAVEL           1  
..                    ...         ...         ...  
725                    뉴욕     OUTDOOR           0  
726                   푸나카     CULTURE           0  
727                    켄팅     OUTDOOR           0  
728                    세부      TRAVEL           0  
729                   NaN         NaN           0  

[730 rows x 7 columns]

# 2.데이터 크롤링 시작

In [5]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('--incognito')
prefs = {'intl.accept_languages': 'en,en_US','cookies': 2, 'images': 1, 'plugins': 2, 'popups': 2,
                                                    'geolocation': 2, 'notifications': 2,
                                                    'auto_select_certificate': 2, 'fullscreen': 2, 'mouselock': 2,
                                                    'mixed_script': 2, 'media_stream': 2, 'media_stream_mic': 2,
                                                    'media_stream_camera': 2, 'protocol_handlers': 2,
                                                    'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2,
                                                    'push_messaging': 2, 'ssl_cert_decisions': 2,
                                                    'metro_switch_to_desktop': 2, 'protected_media_identifier': 2,
                                                    'app_banner': 2, 'site_engagement': 2, 'durable_storage': 2}
# options.add_experimental_option('prefs',prefs )
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")

drv       = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
data_list = [] 
drv.get('https://www.youtube.com/')

/var/folders/tp/dglqnwk96sbbcnl6wq3tl3k00000gn/T/ipykernel_58269/3226614437.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  drv       = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
/var/folders/tp/dglqnwk96sbbcnl6wq3tl3k00000gn/T/ipykernel_58269/3226614437.py:18: DeprecationWarning: use options instead of chrome_options
  drv       = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [ ]:
for idx, row in tqdm(data[0:].iterrows(), total = data[0:].shape[0], desc="돌리는중"):

    cd_idx = row['cd_idx']
    search_keyword = row['bd_name']
    print(search_keyword)
    drv.get('https://www.youtube.com/results?search_query=' + str(search_keyword))
    
    
    bd_youtube_search_ko = search_keyword
    no_of_pagedowns = 5
    elem = drv.find_element(By.TAG_NAME, "body")

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.2)
        no_of_pagedowns -= 1
    time.sleep(5)  # sleep을 통해 Robot Error 해결.
    soup=BeautifulSoup(drv.page_source, "html")

    Alll=soup.findAll("ytd-video-renderer", {"class":"style-scope ytd-item-section-renderer"})
    if len(Alll) == 0 :
        data_list.append({"cd_idx": cd_idx})
    '#metadata-line > span:nth-child(2)' # view
    'inline-metadata-item style-scope ytd-video-meta-block'# view span class
    '#video-title > yt-formatted-string' # title
    num = 0
    for i in Alll :
        try :
            title =i.find("a",{"id":"video-title"})['title']
        except :
            title = None
        try:
            link = 'https://www.youtube.com' + i.find("a",{"id":"video-title"})['href']
        except:
            link = None
        try:
#             text = i.find("yt-formatted-string",{"id" : "description-text"}).text
            text = i.find("yt-formatted-string",{"class" : "metadata-snippet-text style-scope ytd-video-renderer"}).text            
        except:
            text = None
        try:
            image=i.find('img',{"id":"img"})["src"]

        except:
            print(f"image크롤링오류: {cd_idx}:{image}")
            image = None
        
        try:
            name = i.find('yt-formatted-string',{"class":"style-scope ytd-channel-name complex-string"}).text
        except:
            name = None
            print('name: error')
            
        try :
            duration=i.find('span',{"class":"style-scope ytd-thumbnail-overlay-time-status-renderer"}).text.strip()
        except :
            duration = None
            print('duration: error')
        try :
            viewCount_raw = i.findAll('span',{"class":"inline-metadata-item style-scope ytd-video-meta-block"})[0].text
        except :
            viewCount_raw = None
            print('viewCount_raw: error')
        try :
            regdate_ment = i.findAll('span',{"class":"inline-metadata-item style-scope ytd-video-meta-block"})[1].text
        except :
            regdate_ment = None
            print('regdate_ment: error')
        dataii = {"cd_idx": cd_idx, "bd_youtube_search_ko":search_keyword, "link": link, "title": title,
        "text": text, "image": image, "name": name,
        "duration": duration, "regdate_ment": regdate_ment, "viewCount_raw": viewCount_raw}
        data_list.append(dataii)
        num += 1
        if (num == 10):
            break
            
#제출날짜 및 세트 번호 수정
drv.close()

In [9]:
data_list

[{'cd_idx': 3193035,
  'bd_youtube_search_ko': "Take a walk in Harry Potter's magician street, Diagon Alley",
  'link': 'https://www.youtube.com/watch?v=PjgJZCKaVdI',
  'title': "Harry Potter and the Sorcerer's Stone (2001) - Diagon Alley Scene",
  'text': "FILM DESCRIPTION: Harry Potter and the Sorcerer's Stone is a 2001 fantasy film directed by Chris Columbus and distributed by\xa0...",
  'image': 'https://yt3.ggpht.com/LNf91hjJUCURqg2pNOKEot-QAZ4GQpcCKDLFJfmnJTK159SRgZOZ52jwgmEw_1oQufkftz2W=s68-c-k-c0x00ffffff-no-rj',
  'name': 'Harry Potter Things',
  'duration': '3:39',
  'regdate_ment': '1년 전',
  'viewCount_raw': '조회수 4.3만회'},
 {'cd_idx': 3193035,
  'bd_youtube_search_ko': "Take a walk in Harry Potter's magician street, Diagon Alley",
  'link': 'https://www.youtube.com/watch?v=1LUNshGrLrk',
  'title': 'Diagon Alley (Relaxing Walk) Universal Studios Florida | Harry Potter World Full Walking Tour 2020',
  'text': 'Welcome back to Universal Studios in Orlando, Florida. We are contin

In [11]:
columns = [
    'cd_idx', 'bd_youtube_search_ko', 'link', 'title',
    'text', 'image', 'name',  'duration', 'regdate_ment', 'viewCount','viewCount_raw','ko_youtube'] 
result_df = pd.DataFrame(data_list, columns=columns)

In [12]:
result_df

cd_idx                               bd_youtube_search_ko  \
0   3193035  Take a walk in Harry Potter's magician street,...   
1   3193035  Take a walk in Harry Potter's magician street,...   
2   3193035  Take a walk in Harry Potter's magician street,...   
3   3193035  Take a walk in Harry Potter's magician street,...   
4   3193035  Take a walk in Harry Potter's magician street,...   
..      ...                                                ...   
65  3193042  Enjoy the masterpiece of the Renaissance, Duom...   
66  3193042  Enjoy the masterpiece of the Renaissance, Duom...   
67  3193042  Enjoy the masterpiece of the Renaissance, Duom...   
68  3193042  Enjoy the masterpiece of the Renaissance, Duom...   
69  3193042  Enjoy the masterpiece of the Renaissance, Duom...   

                                           link  \
0   https://www.youtube.com/watch?v=PjgJZCKaVdI   
1   https://www.youtube.com/watch?v=1LUNshGrLrk   
2    https://www.youtube.com/shorts/UbkLZ-9GZTs   
3   https://www.youtube.com/watch?v=hBZAv-a7crQ   
4   https://www.youtube.com/watch?v=vMooEWFeerM   
..                                          ...   
65  https://www.youtube.com/watch?v=Dt_6rhLVgwo   
66  https://www.youtube.com/watch?v=6y_eL_0sQQw   
67  https://www.youtube.com/watch?v=KgNCNN4fWKA   
68  https://www.youtube.com/watch?v=OZPnFL4e6Ik   
69  https://www.youtube.com/watch?v=9QXtGY_do8E   

                                                title  \
0   Harry Potter and the Sorcerer's Stone (2001) -...   
1   Diagon Alley (Relaxing Walk) Universal Studios...   
2   Harry Potter Shop in Edinburgh, Victoria Stree...   
3   Harry Potter London Magic Walking Tour and Fil...   
4   Argh so bright, mah eyes! Weasley's Wizard Whe...   
..                                                ...   
65  The Apex of High Renaissance, Raphael Sanzio's...   
66                       History Summarized: Florence   
67   7 Masterpieces Not to Miss at the Uffizi Gallery   
68  Duomo (Cathedral) Complex -- Florence Art Tour...   
69  Duomo in Florence Story | Principles of Excell...   

                                                 text  \
0   FILM DESCRIPTION: Harry Potter and the Sorcere...   
1   Welcome back to Universal Studios in Orlando, ...   
2   According to Google maps Victoria Street is th...   
3   Harry Potter London Magic Walking Tour and Fil...   
4   Do orange and purple go together? I'm not exac...   
..                                                ...   
65  Raphael Sanzio was one of the top three master...   
66  Can't start a Renaissance without building a f...   
67  Calling all art lovers! Located in the heart o...   
68  A summary of the art and visiting options arou...   
69  The importance of hard work is highlighted thr...   

                                                image  \
0   https://yt3.ggpht.com/LNf91hjJUCURqg2pNOKEot-Q...   
1   https://yt3.ggpht.com/ytc/AMLnZu-Qg7t6zytsMh9i...   
2   https://yt3.ggpht.com/wh4dP5GPIN84g0ncbf6K3bI1...   
3   https://yt3.ggpht.com/ytc/AMLnZu--dQUjg1hNaCvp...   
4   https://yt3.ggpht.com/-ERITRszY8GZcyVpv4f_0fx1...   
..                                                ...   
65  https://yt3.ggpht.com/WNdkfI56ix0kgMYUzM-ycBd5...   
66  https://yt3.ggpht.com/ytc/AMLnZu-Evv_YVYi9P9my...   
67  https://yt3.ggpht.com/ytc/AMLnZu9p2OilEabSGRyd...   
68  https://yt3.ggpht.com/ytc/AMLnZu-3Y_A_8t4bOH0u...   
69  https://yt3.ggpht.com/ytc/AMLnZu8Q8IGOIusVLs3P...   

                                                 name duration regdate_ment  \
0                                 Harry Potter Things     3:39         1년 전   
1                                    Theme Park Shark     9:08         2년 전   
2                                   Rhona The Unicorn   SHORTS         2주 전   
3   Joolz Guides - London History Walks - Travel F...    17:25         5년 전   
4                                             ellieV     30:15         2년 전   
..                                                ...      ...        

In [ ]:
result_df = pd.DataFrame(data_list, columns=columns)
result_df

In [9]:
result_df

cd_idx                               bd_youtube_search_ko  \
0     3193035  Take a walk in Harry Potter's magician street,...   
1     3193035  Take a walk in Harry Potter's magician street,...   
2     3193035  Take a walk in Harry Potter's magician street,...   
3     3193035  Take a walk in Harry Potter's magician street,...   
4     3193035  Take a walk in Harry Potter's magician street,...   
...       ...                                                ...   
7250        ↵  판단 잎으로 만드는 달콤함과 고소함의 결정체, 카야잼을 아시나요? 싱가포르의 '야쿤...   
7251        ↵  판단 잎으로 만드는 달콤함과 고소함의 결정체, 카야잼을 아시나요? 싱가포르의 '야쿤...   
7252        ↵  판단 잎으로 만드는 달콤함과 고소함의 결정체, 카야잼을 아시나요? 싱가포르의 '야쿤...   
7253        ↵  판단 잎으로 만드는 달콤함과 고소함의 결정체, 카야잼을 아시나요? 싱가포르의 '야쿤...   
7254        ↵  판단 잎으로 만드는 달콤함과 고소함의 결정체, 카야잼을 아시나요? 싱가포르의 '야쿤...   

                                             link  \
0     https://www.youtube.com/watch?v=PjgJZCKaVdI   
1     https://www.youtube.com/watch?v=0g1PPDlKZlU   
2     https://www.youtube.com/watch?v=QobZIsKd-68   
3     https://www.youtube.com/watch?v=ZBSpbtVLObc   
4     https://www.youtube.com/watch?v=wXh1ae8zOr0   
...                                           ...   
7250  https://www.youtube.com/watch?v=obvBZlP5FMg   
7251  https://www.youtube.com/watch?v=0WTPw0-15k4   
7252  https://www.youtube.com/watch?v=WiQ36gtNVyA   
7253  https://www.youtube.com/watch?v=4FxxsGsgwww   
7254  https://www.youtube.com/watch?v=or1LUqCj6yk   

                                                  title  \
0     Harry Potter and the Sorcerer's Stone (2001) -...   
1     Top 6 Diagon Alley Hidden Gems | Secrets of Di...   
2     Diagon Alley | Harry Potter Inspired Music | A...   
3     Diagon Alley at Leadenhall Market London, Harr...   
4                           Take part in Walk with Joe!   
...                                                 ...   
7250  🅾️다진생강.다진마늘 냉동하기(초코볼 담는 플라스틱통 활용하기) [How to Fr...   
7251  수정과의 단짝! 달달, 쫀득, 고소한 ‘곶감쌈’만들기/ 干し柿のくるみ巻き/ Gotg...   
7252  집밥숨선생으로 거듭나기💫 / 운동하고 밥먹고 / 대구 수목원 / 빠다롤더테라스 / ...   
7253  [모의고사] 식물 분류 동정 (14) / 피자식물(목본) 20종 / #청미래덩굴과#...   
7254  #49 [매일패턴] 패턴으로 배우는 영어표현/ Don't you have to te...   

                                                   text  \
0     FILM DESCRIPTION: Harry Potter and the Sorcere...   
1     The best of Diagon Alley, let's explore the to...   
2     Take a trip down Diagon Alley with beautiful m...   
3     Diagon Alley is one of many important location...   
4     Join in with Joe Wicks and Walk with Joe this ...   
...                                                 ...   
7250  다진생강.다진마늘 냉동하기(초코볼 담는 플라스틱통 활용하기) [How to Free...   
7251  구독&좋아요 부탁드려요(_ _) 감사합니다! 수정과의 단짝! '곶감쌈'을 만들어 보...   
7252  5일간의 일상입니다♪ *BGM - Music by carego - Little Ch...   
7253  생물분류기사 #생물분류 #식물분류 #생물 #식물 #피자식물 #속씨식물 #청미래덩굴과...   
7254  이제 출발해야 되지 않니? Don't you have to leave now? 가라...   

                                                  image                  name  \
0     https://i.ytimg.com/vi/PjgJZCKaVdI/hq720.jpg?s...   Harry Potter Things   
1     https://i.ytimg.com/vi/0g1PPDlKZlU/hq720.jpg?s...              Rix Flix   
2     https://i.ytimg.com/vi/QobZIsKd-68/hq720.jpg?s...   Ambient Soundscapes   
3     https://i.ytimg.com/vi/ZBSpbtVLObc/hq720.jpg?s...       KEEP WALKING 4K   
4     https://i.ytimg.com/vi/wXh1ae8zOr0/hq720.jpg?s...  BBC Children in Need   
...                                                 ...                   ...   
7250  https://i.ytimg.com/vi/obvBZlP5FMg/hq720.jpg?s...               soi소이할미   
7251  https://i.ytimg.com/vi/0WTPw0-15k4/hq720.jpg?s...         장보심 jangbosim   
7252  https://i.ytimg.com/vi/WiQ36gtNVyA/hqdefault.j...     프롬숨숨 From Sumsumm   
7253  https://i.ytimg.com/vi/4FxxsGsgwww/hqdefault.j...        은재맘 ' 나무 공부방 '   
7254  https://i.ytimg.com/vi/or1LUqCj6yk/hqdefault.j...            참쉬운사다리5분영어   

     duration  regdate_ment  viewCount  viewCount_raw  ko_youtube  
0        3:39           NaN        NaN            NaN         NaN  
1       10:32           Na

# 3. 최종 데이터 정제 저장

In [10]:
# 임시 백업 df 설정
temp_df = pd.DataFrame(data_list, columns=columns)

# 결측값 있는 행 확인
check_columns  = ["cd_idx", "bd_youtube_search_ko", "link", "title", "image", "name"]
missing_values = result_df[result_df[check_columns].isnull().any(axis=1)]

print(len(temp_df))
print(len(missing_values))
print(len(temp_df) - len(missing_values))

7255
23
7232


In [11]:
# 결측값 있는 행 drop 및 저장
result_df.drop(missing_values.index, axis=0, inplace=True)
print(f"1차 drop 이후 개수: {len(result_df)}")

# 추가 요청 사항 drop 및 저장
drop_df    = result_df[(result_df["duration"].isnull() == True) | (result_df["regdate_ment"].isnull() == True)]
drop_index = drop_df.index
result_df.drop(drop_index, axis=0, inplace=True)
print(f"추가 drop 개수: {len(drop_index)}")

# index 정렬
result_df.reset_index(drop=True, inplace=True)
print(len(result_df))

1차 drop 이후 개수: 7232
추가 drop 개수: 7232
0


# 4.나머지 정보 채우기 -viewCount

In [12]:
for idx in tqdm(range(len(result_df))): 
    raw=result_df['viewCount_raw'][idx].replace(' views','')
    if 'K' in raw:
        raw=float(raw.replace('K',''))*1000
    elif 'M' in raw:
        raw=float(raw.replace('M',''))*1000000
    elif 'B' in raw:
        raw=float(raw.replace('B',''))*1000000000
    
    elif 'No' in raw:
        raw=0
    elif 'view' in raw:
        raw=result_df['viewCount_raw'][idx].replace(' view','')
    raw=int(raw)
    result_df['viewCount'][idx]=raw
    print(raw,type(raw))

0it [00:00, ?it/s]


# excel 파일로 저장

In [14]:
Desktop

'/Users/mycelebs/Desktop/staypia_데이터수집'

In [15]:
result_df.to_csv(f"{Desktop}/youtube/csv_result/youtube_crawling_bucketlist_" + datetime.datetime.now().strftime('%y%m%d') +"-en.csv")